In [1]:
import logging
import numpy as np
from sentence_transformers import SentenceTransformer
from weighted_bert.models import WeightedAverage, WeightedRemoval

logging.basicConfig(level="INFO")

%load_ext autoreload
%autoreload 2

# Example use

In [2]:
weighting_checkpoint = "savasy/bert-base-turkish-ner-cased"
embedding_checkpoint = "sentence-transformers/distiluse-base-multilingual-cased-v1"

documents = [
[
    "Tesla'nın otomobilleri insan hayatlarını riske atıyor olabilir.",
    "Türkiye ve Kore arasında gerçekleşen voleybol müsabakasını Türkiye Milli Takımı kazandı.",
    "Bu bir metin.",
],
[
    "Mustafa Kemal Atatürk 19 Mayıs 1919'da Samsun'a ayak bastı.",
    "Bu bir metin.",
],
]

In [3]:
# Initialize models
embedding_model = SentenceTransformer(embedding_checkpoint)
weighter_a = WeightedAverage(weighting_checkpoint)
weighter_r = WeightedRemoval(weighting_checkpoint)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/distiluse-base-multilingual-cased-v1
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
INFO:weighted_bert.models:================ Loading weighting model ================
INFO:weighted_bert.models:	savasy/bert-base-turkish-ner-cased
/home/emrecan/workspace/weighted_bert/venv/lib/python3.8/site-packages/transformers/pipelines/token_classification.py:128: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(
INFO:weighted_bert.models:================ Loading weighting model ================
INFO:weighted_bert.models:	savasy/bert-base-turkish-ner-cased


In [4]:
# Calculate embeddings
collection_sentence_embeddings = [embedding_model.encode(doc) for doc in documents]
embeddings_a = np.array([weighter_a.get_document_embedding(doc, sentence_emb)
                for doc, sentence_emb in zip(documents, collection_sentence_embeddings)])
embeddings_r = weighter_r.get_document_embeddings(documents, collection_sentence_embeddings)

Batches: 100%|██████████| 1/1 [00:00<00:00, 94.74it/s]
/home/emrecan/workspace/weighted_bert/venv/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
INFO:weighted_bert.models:================ Detecting entities ================
INFO:weighted_bert.models:================ Calculating initial document embeddings ================
INFO:weighted_bert.models:================ Correcting document embeddings ================
INFO:weighted_bert.models:	Calculating first singular vector...
INFO:weighted_bert.models:	Calculating corrected embeddings...


In [5]:
embeddings_a.shape

(2, 512)

In [6]:
embeddings_r.shape

(2, 512)